In [ ]:
!pip install -U sentence-transformers
!pip install tensorflow_decision_forests
from sentence_transformers import SentenceTransformer
import string
import re
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from nltk.corpus import stopwords
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error, r2_score
import nltk
nltk.download('stopwords')

2024-01-02 13:04:04.636653: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 13:04:04.636823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 13:04:04.986861: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 13:04:06.466655: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-02 13:04:15.371184: W tensorflow/compiler/tf2

True

In [ ]:
df_train_prompt = pd.read_csv('prompts_train.csv')
df_train_summaries = pd.read_csv('summaries_train.csv')

In [ ]:
df_train = df_train_summaries.merge(df_train_prompt, on='prompt_id')

In [ ]:
import vertexai
vertexai.init()

from vertexai.language_models import TextEmbeddingModel

text_embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

In [ ]:
# Count the stop words in the text.
def count_stopwords(text: str) -> int:
    stopword_list = set(stopwords.words('english'))
    words = text.split()
    stopwords_count = sum(1 for word in words if word.lower() in stopword_list)
    return stopwords_count

# Count the punctuations in the text.
# punctuation_set -> !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~
def count_punctuation(text: str) -> int:
    punctuation_set = set(string.punctuation)
    punctuation_count = sum(1 for char in text if char in punctuation_set)
    return punctuation_count

# Count the digits in the text.
def count_numbers(text: str) -> int:
    numbers = re.findall(r'\d+', text)
    numbers_count = len(numbers)
    return numbers_count

# This function applies all the above preprocessing functions on a text feature.
def feature_engineer(dataframe: pd.DataFrame, feature: str = 'text') -> pd.DataFrame:
    dataframe[f'{feature}_word_cnt'] = dataframe[feature].apply(lambda x: len(x.split(' ')))
    dataframe[f'{feature}_length'] = dataframe[feature].apply(lambda x: len(x))
    dataframe[f'{feature}_stopword_cnt'] = dataframe[feature].apply(lambda x: count_stopwords(x))
    dataframe[f'{feature}_punct_cnt'] = dataframe[feature].apply(lambda x: count_punctuation(x))
    dataframe[f'{feature}_number_cnt'] = dataframe[feature].apply(lambda x: count_numbers(x))
    return dataframe

preprocessed_df = feature_engineer(df_train)

In [ ]:
df_train['cosine_similarity'] = np.nan
df_train['prompt_text_embeddings'] = np.empty((len(df_train),), dtype=object)
df_train['summary_text_embeddings'] = np.empty((len(df_train),), dtype=object)

for index, row in df_train.iterrows():

    prompt_text_embeddings = text_embedding_model.get_embeddings([row['prompt_text']])[0].values
    summary_text_embeddings = text_embedding_model.get_embeddings([row['text']])[0].values

    df_train.at[index, 'prompt_text_embeddings'] = prompt_text_embeddings
    df_train.at[index, 'summary_text_embeddings'] = summary_text_embeddings

    prompt_text_embeddings = np.array(prompt_text_embeddings).reshape(1, -1)
    summary_text_embeddings = np.array(summary_text_embeddings).reshape(1, -1)


    cosine_similarity_result = cosine_similarity(prompt_text_embeddings,summary_text_embeddings)

    df_train.at[index, 'cosine_similarity'] = cosine_similarity_result

    # print(index)

    # if you need to track the progress, uncomment the following two lines
    if (index % 3700 == 0 and index>0):
        time.sleep(70)
    if (index % 100 == 0):
        print(index)

df_train.to_pickle('df_train.pkl')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
